# i.MX 95 AHAB with U-BOOT

This notebook describes how to build a bootable AHAB image with ELE firmware and U-BOOT bootloader, and how to use NXPELE app.

> **Note**: This guide was updated for silicon revision B0 that uses AHAB container version 2. If sillicon revision is A0 or A1, change it in the config files

## 1. Prerequisites

- SPSDK is needed with examples extension. `pip install spsdk[examples]` (Please refer to the installation documentation.)
- This demo was tested with i.MX 95 EVK board with LPDDR5 memory and A0 chip revision.

## 1.1 Images preparation

- to create resulting binary containing AHAB containers, we need to prepare the binaries
- in this section we reproduce the process which is done by the `imx-mkimage` tool
- Obtain all the necessary binaries and put them into inputs directory

## 1.2 U-Boot

In order to use the nxpele app, U-Boot must be built with AHAB support. CONFIG_AHAB_BOOT=y
If you want to use the nxpele over fastboot, also multiplexing of console output to fastboot must be enabled by setting CONFIG_CONSOLE_MUX=y.

## 1.3 Requirements

Prepare YOCTO build and copy files into the inputs directory.

Primary image container set:
- ELE firmware
- lpddr5 or lpddr4 firmware files with OEI firmware
- CM33 OEI TCM
- CM33 System manager image
- U-Boot SPL
- [Optional] M7 application image

Secondary image container set
- bl31.bin binary (ARM Trusted Firmware)
- U-Boot (built with AHAB support)
- TEE binary

In [1]:
from spsdk.utils.jupyter_utils import YamlDiffWidget

# This env variable sets colored logger output to STDOUT
# Execute this cell to enable execution of the ! line magic
%env JUPYTER_SPSDK=1
%alias execute echo %l && %l
%alias_magic ! execute

env: JUPYTER_SPSDK=1
Created `%!` as an alias for `%execute`.


## 2. AHAB Image


### 2.1 AHAB Template

We can generate the template using the nxpimage ahab get-template command. The command will generate a template. The template is a YAML file that contains the AHAB header and the AHAB container. The AHAB header contains the information about the image, such as the version, the number of containers, and the signature. The AHAB container contains the information about the image, such as the image type, the image version, the image size, and the image signature.

The following command generates the template:

```bash
nxpimage ahab get-template -f mimx9596 -o ahab_template.yaml
```

### 2.2 Exporting of the AHAB image

The AHAB image can be exported using the nxpimage ahab export command. The command will create the AHAB image from the template. The following command creates the AHAB image:

```bash
nxpimage ahab export -c ahab_template.yaml 
```


### 2.4 Primary image container set

Primary image container set consists of the following images:

- ELE firmware
- DDR firmware with OEI
- System manager
- U-Boot SPL
- [Optional] M7 application image

In [2]:
YamlDiffWidget("inputs/mx95_ahab_primary.diffc").html

nxpimage ahab get-template -r a0 -f mimx9596 -o workspace/ahab_template.yaml --force 
Creating workspace/ahab_template.yaml template file.


### 2.5 Secondary image container set

Secondary image container set consists of the following images:

- bl31.bin binary (ARM Trusted Firmware)
- U-Boot (built with AHAB support)
- TEE binary


In [3]:
YamlDiffWidget("inputs/mx95_ahab_secondary.diffc").html

nxpimage ahab get-template -r a0 -f mimx9596 -o workspace/ahab_template.yaml --force 
Creating workspace/ahab_template.yaml template file.


### 2.6 Exporting of full AHAB image (container set)

U-Boot image consist from two image container sets. The first one contains ELE firmware, DDR firmware with OEI, system manager and U-Boot SPL. This image is loaded using the SDPS protocol. Once the U-Boot SPL is loaded to OCRAM the fastboot is opened and the second container is loaded using the fastboot protocol.

If the image is intended for loading using the UUU, the memory type should be set to serial_downloader. However the type of each individual AHAB should be set to *standard*, because it will be stored in flash memory.

```bash
nxpimage -v bootable-image export -c u-boot-flash_template.yaml -o flash.bin
```



In [4]:
YamlDiffWidget("inputs/mx95_ahab_bimg.diffc").html

nxpimage bootable-image get-templates -f mimx9596 -o workspace --force 
Creating workspace/bootimg_mimx9596_serial_downloader.yaml template file.
Creating workspace/bootimg_mimx9596_flexspi_nor.yaml template file.
Creating workspace/bootimg_mimx9596_emmc.yaml template file.
Creating workspace/bootimg_mimx9596_sd.yaml template file.
Creating workspace/bootimg_mimx9596_recovery_spi.yaml template file.


In [5]:
U_BOOT_PRIMARY = "inputs/mx95_uboot_primary.yaml"
U_BOOT_SECONDARY = "inputs/mx95_uboot_secondary.yaml"
U_BOOT_FLASH_BOOT_CFG = "inputs/bootimg_mx95_serial_downloader.yaml"
U_BOOT_FLASH_BOOT = "outputs/flash.bin"

VERBOSITY = "-v"

# EXPORT U-BOOT IMAGE
%! nxpimage -v bootable-image export --config $U_BOOT_FLASH_BOOT_CFG --output $U_BOOT_FLASH_BOOT

nxpimage -v bootable-image export --config inputs/bootimg_mx95_serial_downloader.yaml --output outputs/flash.bin
INFO:spsdk.apps.nxpimage_apps.nxpimage_bimg:Created Bootable Image:
Name:      Bootable Image for mimx9596, Revision: latest
Starts:    0x0
Ends:      0x2923ff
Size:      Size: 2.6 MiB; 2,696,192 B
Alignment: 1 B
Execution Start Address: Not defined
Pattern:zeros
Memory type: MemoryType.SERIAL_DOWNLOADER

INFO:spsdk.apps.nxpimage_apps.nxpimage_bimg:Created Bootable Image memory map:

┌──0x0000_0000─ Bootable Image for mimx9596, Revision: latest ───┐
│                   Size: 2.6 MiB; 2,696,192 B                   │
│           Memory type: MemoryType.SERIAL_DOWNLOADER            │
│                         Pattern: zeros                         │
│┌──0x0000_0000─ primary_image_container_set ───────────────────┐│
││                  Size: 912.0 kiB; 933,888 B                  ││
││            AHAB Image for mimx9596, Revision: b0             ││
││                        Patte

In [6]:
# Set the boot mode to Cortex-M Serial Downloader 1001 and download the files using the UUU tool

%! nxpuuu $VERBOSITY write -f mimx9596 -b emmc $U_BOOT_FLASH_BOOT

nxpuuu -v write -f mimx9596 -b emmc outputs/flash.bin 
SDPS: boot -f outputs/flash.bin


SDPV: write -f outputs/flash.bin -skipspl


SDPV: jump


Done


Now change the boot mode to Cortex-M EMMC 1010 and reset the board. Find the serial port that belongs to U-Boot console and interrupt the boot. 
When the console is switched to U-Boot menu, we can use the *nxpele* tool to communicate with the ELE.


In [7]:
%! nxpele -f mimx9596 -p /dev/tty.usbserial-133202 -d uboot_serial get-info

nxpele -f mimx9596 -p /dev/tty.usbserial-133202 -d uboot_serial get-info
ELE get info ends successfully:
Command:          0xda
Version:          3
Length:           256
SoC ID:           MX95 - 0x9500
SoC version:      B000
Life Cycle:       OEM_OPEN - 0x0010
SSSM state:       4
UUID:             f42b00351ce84cc29685689681dab20e
SHA256 ROM PATCH: 72d02b666a524aca0a3162accb4c8de7af33083fc2faefb249d87c7de1437c81
SHA256 FW:        065165a3ed9dc3a6a5243d32a7fb57c9b50cfd29fa18ca0f2aed18f5d7fb7038



In [8]:
%! nxpele -f mimx9596 -p /dev/tty.usbserial-133202 -d uboot_serial generate-keyblob DEK -a AES_CBC --key-id 0 --key 00000000000000000000000000000000 --key-size 128

nxpele -f mimx9596 -p /dev/tty.usbserial-133202 -d uboot_serial generate-keyblob DEK -a AES_CBC --key-id 0 --key 00000000000000000000000000000000 --key-size 128
ELE generate DEK key blob ends successfully:
00480081011003006fc6e2aa338c6bddfa8a4dd42b324872073222002fb5974b045cb47bb6c718ef402b3f0aeeb5a252f7174787f7ba4c8983bd158979870a378a581d50d0527dec
